In [1]:
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt
import os
import swnMetrics as swn


import helperFunc as hf

## Functions

In [2]:
def getModuAll(adjDict,taus,pRandRewires,repetitions = 100):
    
    lenTaus = len(taus)
    lenPRand = len(pRandRewires)
    Q = np.zeros((lenPRand,lenTaus,repetitions))  
    
    for rep in np.arange(repetitions):
        for indT,tau in enumerate(taus):
            for indP,p in enumerate(pRandRewires):
        
                #load the specific rewired matrix
                A = adjDict[rep+1][(p, tau, 4000)][1]
        
                #construct it so that igraph can read it
                #make it undirected
                g = ig.Graph(directed=False)
                #make it weighted
                g.es["weight"] = 1.0                
                
                g.add_vertices(len(A))
                ix, jx = A.nonzero()
                for i,j in zip(ix, jx):
                    if i<j:
                        g.add_edge(i, j, weight=A[i,j])
            
                #calculate the clusters and their modularity score    
                clusters = g.community_multilevel(weights=g.es['weight'])
                modularity_score = g.modularity(clusters.membership,weights=g.es['weight'])
        
                #store it in the matrix
                Q[indP,indT,rep] = modularity_score
                
    return Q

## Manipulate data to the the Q matrices and to save them -- LOGNORMAL

In [44]:
#parameters tested
rewirings = 4000
pRand = [0,0.2]

taus = [5.4,5.45,5.5,5.55,5.6,5.65,5.7,5.75,5.8,5.85,5.9,5.95,6]
weightDist = 'lognormal'

###### Load Adjacency matrices
directoryALoad ='data/ArandA/500iterations/'

#filePathALoad = directoryALoad + 'ArandA_tauTransition500_'+weightDist+'_'+str(rewirings)+'.pckl'
filePathALoad = directoryALoad + 'ArandA_tauTransition250firstHalf_'+weightDist+'_'+str(rewirings)+'.pckl'
filePathALoad2 = directoryALoad + 'ArandA_tauTransition250secondHalf_'+weightDist+'_'+str(rewirings)+'.pckl'

ArandA = hf.loadVar(filePathALoad)
ArandA2 = hf.loadVar(filePathALoad2)


In [46]:
Q1 = getModuAll(ArandA,taus,pRand,250)
Q2 = getModuAll(ArandA2,taus,pRand,250)


In [47]:
Q = np.concatenate((Q1, Q2), axis=2)
Q2.shape

(2, 13, 250)

In [48]:

directoryQSave ='data/ModularityValues/500iterations/'
descr = 'QTransition_'
filePathQSave = directoryQSave + descr+weightDist+'.pckl'
hf.saveVarSimple((Q,taus), filePathQSave)

## Normal -get the Q

In [3]:
rewirings = 4000
pRand = [0,0.2]
taus = np.array([4.0,4.05,4.1,4.15,4.2,4.25,4.3])

directoryLoad ='data/ArandA/500iterations/'
weightDist = 'normal'

filePathLoad = directoryLoad + 'ArandA_tauTransition_'+weightDist+'_'+str(rewirings)+'.pckl'

ArandA = hf.loadVar(filePathLoad)

In [4]:
#Calculate the modularity values
Q = getModuAll(ArandA,taus,pRand,500)

In [5]:
directoryQSave ='data/ModularityValues/500iterations/'
descr = 'QTransition_'
filePathQSave = directoryQSave + descr+weightDist+'.pckl'
hf.saveVarSimple((Q,taus), filePathQSave)

In [6]:
Q.shape

(2, 7, 500)

In [8]:
Q[:,:,100]

array([[0.67874775, 0.61435013, 0.45723332, 0.33631418, 0.66121412,
        0.13302398, 0.14351698],
       [0.26630917, 0.65997187, 0.59883586, 0.3115136 , 0.2980781 ,
        0.16075964, 0.4397877 ]])

## Add the 0 variation to tau var AArand

In [44]:
weightDist = 'normal'
p =  0.2

tauSpread = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.,1.1,1.2,1.3,1.4,1.5]
#tauCenter = [5.8]*len(tauSpread)
#tauCenter = [5.5]*len(tauSpread)
#tauCenter = [4.2]*len(tauSpread)
tauCenter = [4.15]*len(tauSpread)

#tauDistribution = ['normal']*len(tauSpread)
tauDistribution = ['uniform']*len(tauSpread)

tauTuples = list(zip(tauCenter,tauSpread,tauDistribution))

directoryVarLoad ='data/ArandA/variableTau/'
filePathVarLoad = directoryVarLoad + 'ArandA_tauTransition_'+weightDist+'_pRand'+str(p)+tauDistribution[0]+'.pckl'

tauTarget = tauCenter[0]
directoryLoad = 'data/ArandA/'
rewirings = 4000
filePathLoad = directoryLoad + 'ArandA_tauTransition_'+weightDist+'_'+str(rewirings)+'.pckl'


In [45]:
ArandAVar = hf.loadVar(filePathVarLoad)

ArandA = hf.loadVar(filePathLoad)

In [46]:
tausIter = [tauCenter[0]]*4000
for it in ArandA.keys():
    kk = (ArandA[it][p,tauCenter[0],4000][0],ArandA[it][p,tauCenter[0],4000][1],tausIter)
    ArandAVar[it][p,(tauCenter[0],0,tauDistribution[0]),4000] = kk


In [47]:
ArandAVar[2].keys()

dict_keys([(0.2, (4.15, 0.1, 'uniform'), 4000), (0.2, (4.15, 0.2, 'uniform'), 4000), (0.2, (4.15, 0.3, 'uniform'), 4000), (0.2, (4.15, 0.4, 'uniform'), 4000), (0.2, (4.15, 0.5, 'uniform'), 4000), (0.2, (4.15, 0.6, 'uniform'), 4000), (0.2, (4.15, 0.7, 'uniform'), 4000), (0.2, (4.15, 0.8, 'uniform'), 4000), (0.2, (4.15, 0.9, 'uniform'), 4000), (0.2, (4.15, 1.0, 'uniform'), 4000), (0.2, (4.15, 1.1, 'uniform'), 4000), (0.2, (4.15, 1.2, 'uniform'), 4000), (0.2, (4.15, 1.3, 'uniform'), 4000), (0.2, (4.15, 1.4, 'uniform'), 4000), (0.2, (4.15, 1.5, 'uniform'), 4000), (0.2, (4.15, 0, 'uniform'), 4000)])

In [48]:
ArandA[1].keys()

dict_keys([(0, 3.5, 4000), (0.2, 3.5, 4000), (0, 3.55, 4000), (0.2, 3.55, 4000), (0, 3.6, 4000), (0.2, 3.6, 4000), (0, 3.65, 4000), (0.2, 3.65, 4000), (0, 3.7, 4000), (0.2, 3.7, 4000), (0, 3.75, 4000), (0.2, 3.75, 4000), (0, 3.8, 4000), (0.2, 3.8, 4000), (0, 3.85, 4000), (0.2, 3.85, 4000), (0, 3.9, 4000), (0.2, 3.9, 4000), (0, 3.95, 4000), (0.2, 3.95, 4000), (0, 4, 4000), (0.2, 4, 4000), (0, 4.05, 4000), (0.2, 4.05, 4000), (0, 4.1, 4000), (0.2, 4.1, 4000), (0, 4.15, 4000), (0.2, 4.15, 4000), (0, 4.2, 4000), (0.2, 4.2, 4000), (0, 4.25, 4000), (0.2, 4.25, 4000), (0, 4.3, 4000), (0.2, 4.3, 4000), (0, 4.35, 4000), (0.2, 4.35, 4000), (0, 4.4, 4000), (0.2, 4.4, 4000), (0, 4.45, 4000), (0.2, 4.45, 4000), (0, 4.5, 4000), (0.2, 4.5, 4000)])

In [49]:
hf.saveVarSimple(ArandAVar, filePathVarLoad)

## Normal

In [15]:
#parameters tested
rewirings = 4000
pRand = [0,0.2]

taus1 = np.array([4.1,4.15,4.2,4.25])
taus2 = np.array([4.0,4.05,4.3])

weightDist = 'normal'

###### Load Adjacency matrices
directoryALoad ='data/ArandA/500iterations/'

#filePathALoad = directoryALoad + 'ArandA_tauTransition500_'+weightDist+'_'+str(rewirings)+'.pckl'
filePathALoad = directoryALoad + 'ArandA_tauTransition500_'+weightDist+'_'+str(rewirings)+'.pckl'
filePathALoad2 = directoryALoad + 'ArandA_2tauTransition500_'+weightDist+'_'+str(rewirings)+'.pckl'

ArandA = hf.loadVar(filePathALoad)
ArandA2 = hf.loadVar(filePathALoad2)

In [30]:
ArandA_all = {}
for k in ArandA:
    ArandA_all[k] = {}
    ArandA_all[k].update(ArandA[k])
    ArandA_all[k].update(ArandA2[k])

In [34]:
directorySave ='data/ArandA/500iterations/'

weightDist = 'normal'

filePathSave = directorySave + 'ArandA_tauTransition_'+weightDist+'_'+str(rewirings)+'.pckl'
hf.saveVarSimple(ArandA_all, filePathSave)

## Miscellanious

In [3]:
#taus = np.arange(3.5,4.55,0.05)
#taus = np.array([4.1,4.15,4.2,4.25])
#taus = np.arange(5.35,6.01,0.05)
taus = np.array([4.0,4.05,4.3])
#taus = np.arange(4.5,6.55,0.05)
pRand = [0,0.2]

In [32]:
#parameters tested
rewirings = 4000
pRand = [0,0.2]
taus = [5.4,5.45,5.5,5.55,5.6,5.65,5.7,5.75,5.8,5.85,5.9,5.95,6]


#weightDist = 'normal'
weightDist = 'lognormal'

###### Load Adjacency matrices
directoryALoad ='data/ArandA/500iterations/'


#filePathALoad = directoryALoad + 'ArandA_tauTransition500_'+weightDist+'_'+str(rewirings)+'.pckl'
filePathALoad = directoryALoad + 'ArandA_tauTransition500firstHalf_'+weightDist+'_'+str(rewirings)+'.pckl'

ArandA = hf.loadVar(filePathALoad)

#######Path to save modularity values
#directoryQSave ='data/ModularityValues/'
#descr = 'QAll_'
#descr = 'QTransition_'
#filePathQSave = directoryQSave + descr+weightDist+'.pckl'

In [34]:
kk_actual = list(ArandA[1].keys())

kk_correct = [(0, 5.35, 4000),(0.2, 5.35, 4000),(0, 5.4, 4000),(0.2, 5.4, 4000),(0, 5.45, 4000),(0.2, 5.45, 4000),(0, 5.5, 4000),
 (0.2, 5.5, 4000),(0, 5.55, 4000),(0.2, 5.55, 4000),(0, 5.6, 4000),(0.2, 5.6, 4000),(0, 5.65, 4000),(0.2, 5.65, 4000),
 (0, 5.7, 4000),(0.2, 5.7, 4000),(0, 5.75, 4000),(0.2, 5.75, 4000),(0, 5.8, 4000),(0.2, 5.8, 4000),(0, 5.85, 4000),
 (0.2, 5.85, 4000),(0, 5.9, 4000),(0.2, 5.9, 4000),(0, 5.95, 4000),(0.2, 5.95, 4000),(0, 6, 4000),(0.2, 6, 4000)]

In [35]:
for it in ArandA.keys():
    for ind,k in enumerate(list(ArandA[it].keys())):
        if (np.abs(k[1] - kk_correct[ind][1])<0.01)&(np.abs(k[0] - kk_correct[ind][0])<0.0001): 
            ArandA[it][kk_correct[ind]] = ArandA[it][k]
            del ArandA[it][k]
        else:
            print('there is something wrong')


In [36]:
hf.saveVarSimple(ArandA, filePathALoad)

In [37]:
for k in ArandA[1].keys():
    print(k)

(0, 5.4, 4000)
(0.2, 5.4, 4000)
(0, 5.45, 4000)
(0.2, 5.45, 4000)
(0, 5.5, 4000)
(0.2, 5.5, 4000)
(0, 5.55, 4000)
(0.2, 5.55, 4000)
(0, 5.6, 4000)
(0.2, 5.6, 4000)
(0, 5.65, 4000)
(0.2, 5.65, 4000)
(0, 5.7, 4000)
(0.2, 5.7, 4000)
(0, 5.75, 4000)
(0.2, 5.75, 4000)
(0, 5.8, 4000)
(0.2, 5.8, 4000)
(0, 5.85, 4000)
(0.2, 5.85, 4000)
(0, 5.9, 4000)
(0.2, 5.9, 4000)
(0, 5.95, 4000)
(0.2, 5.95, 4000)
(0, 6, 4000)
(0.2, 6, 4000)


## Miscellanious 2

In [ ]:
list(ArandA[1].keys())[4:]

In [ ]:
kk_correct = [(0, 3.6, 4000),(0.2, 3.6, 4000),(0, 3.65, 4000),(0.2, 3.65, 4000),(0, 3.7, 4000),(0.2, 3.7, 4000),(0, 3.75, 4000),(0.2, 3.75, 4000),
(0, 3.8, 4000),(0.2, 3.8, 4000),(0, 3.85, 4000),(0.2, 3.85, 4000),(0, 3.9, 4000),(0.2, 3.9, 4000),(0, 3.95, 4000),
(0.2, 3.95, 4000),(0, 4, 4000),(0.2, 4, 4000),(0, 4.05, 4000),(0.2, 4.05, 4000),(0, 4.1, 4000),(0.2, 4.1, 4000),
(0, 4.15, 4000),(0.2, 4.15, 4000),(0, 4.2, 4000),(0.2, 4.2, 4000),(0, 4.25, 4000),(0.2, 4.25, 4000),(0, 4.3, 4000),
(0.2, 4.3, 4000),(0, 4.35, 4000),(0.2, 4.35, 4000),(0, 4.4, 4000),(0.2, 4.4, 4000),(0, 4.45, 4000),(0.2, 4.45, 4000),
(0, 4.5, 4000),(0.2, 4.5, 4000)]

In [ ]:
kk_correct = [(0, 4.65, 4000),(0.2, 4.65, 4000),(0, 4.7, 4000),(0.2, 4.7, 4000),(0, 4.75, 4000),(0.2, 4.75, 4000),
(0, 4.8, 4000),(0.2, 4.8, 4000),(0, 4.85, 4000),(0.2, 4.85, 4000),(0, 4.9, 4000),(0.2, 4.9, 4000),(0, 4.95, 4000),
(0.2, 4.95, 4000),(0, 5, 4000),(0.2, 5, 4000),(0, 5.05, 4000),(0.2, 5.05, 4000),(0, 5.1, 4000),(0.2, 5.1, 4000),
(0, 5.15, 4000),(0.2, 5.15, 4000),(0, 5.2, 4000),(0.2, 5.2, 4000),(0, 5.25, 4000),(0.2, 5.25, 4000),(0, 5.3, 4000),
(0.2, 5.3, 4000),(0, 5.35, 4000),(0.2, 5.35, 4000),(0, 5.4, 4000),(0.2, 5.4, 4000),(0, 5.45, 4000),(0.2, 5.45, 4000),
(0, 5.5, 4000),(0.2, 5.5, 4000),(0, 5.55, 4000),(0.2, 5.55, 4000),(0, 5.6, 4000),(0.2, 5.6, 4000),(0, 5.65, 4000),
(0.2, 5.65, 4000),(0, 5.7, 4000),(0.2, 5.7, 4000),(0, 5.75, 4000),(0.2, 5.75, 4000),(0, 5.8, 4000),(0.2, 5.8, 4000),
(0, 5.85, 4000), (0.2, 5.85, 4000),(0, 5.9, 4000),(0.2, 5.9, 4000),(0, 5.95, 4000),(0.2, 5.95, 4000),
(0, 6, 4000),(0.2, 6, 4000),(0, 6.05, 4000),(0.2, 6.05, 4000),(0, 6.1, 4000),(0.2, 6.1, 4000),(0, 6.15, 4000),
(0.2, 6.15, 4000),(0, 6.2, 4000),(0.2, 6.2, 4000),(0, 6.25, 4000),(0.2, 6.25, 4000),(0, 6.3, 4000),(0.2, 6.3, 4000),
(0, 6.35, 4000),(0.2, 6.35, 4000),(0, 6.4, 4000),(0.2, 6.4, 4000),(0, 6.45, 4000),(0.2, 6.45, 4000),(0, 6.5, 4000),
(0.2, 6.5, 4000)]

In [ ]:
for it in ArandA.keys():
    for ind,k in enumerate(list(ArandA[it].keys())[4:]):
        if (np.abs(k[1] - kk_correct[ind][1])<0.01)&(np.abs(k[0] - kk_correct[ind][0])<0.0001): 
            ArandA[it][kk_correct[ind]] = ArandA[it][k]
            del ArandA[it][k]
        else:
            print(k)

In [ ]:
for k in ArandA[1].keys():
    print(k)

In [ ]:
hf.saveVarSimple(ArandA, filePathLoad)

## Fix the tau approximations

In [72]:
rewirings = 4000
pRand = [0,0.2]
  
wD = 'lognormal'
###### Load Adjacency matrices
directoryALoad ='data/ArandA/'

filePathLoad = directoryALoad + 'ArandA_'+wD+'_'+str(rewirings)+'.pckl'
ArandA = hf.loadVar(filePathLoad)    

In [73]:
ArandA[1].keys()

dict_keys([(0, 0, 4000), (0.2, 0, 4000), (0, 0.2, 4000), (0.2, 0.2, 4000), (0, 0.4, 4000), (0.2, 0.4, 4000), (0, 0.6, 4000), (0.2, 0.6, 4000), (0, 0.8, 4000), (0.2, 0.8, 4000), (0, 1, 4000), (0.2, 1, 4000), (0, 1.2, 4000), (0.2, 1.2, 4000), (0, 1.4, 4000), (0.2, 1.4, 4000), (0, 1.6, 4000), (0.2, 1.6, 4000), (0, 1.8, 4000), (0.2, 1.8, 4000), (0, 2, 4000), (0.2, 2, 4000), (0, 2.2, 4000), (0.2, 2.2, 4000), (0, 2.4, 4000), (0.2, 2.4, 4000), (0, 2.6, 4000), (0.2, 2.6, 4000), (0, 2.8, 4000), (0.2, 2.8, 4000), (0, 3, 4000), (0.2, 3, 4000), (0, 3.2, 4000), (0.2, 3.2, 4000), (0, 3.4, 4000), (0.2, 3.4, 4000), (0, 3.6, 4000), (0.2, 3.6, 4000), (0, 3.8, 4000), (0.2, 3.8, 4000), (0, 4, 4000), (0.2, 4, 4000), (0, 4.2, 4000), (0.2, 4.2, 4000), (0, 4.4, 4000), (0.2, 4.4, 4000), (0, 4.6, 4000), (0.2, 4.6, 4000), (0, 4.8, 4000), (0.2, 4.8, 4000), (0, 5, 4000), (0.2, 5, 4000), (0, 5.2, 4000), (0.2, 5.2, 4000), (0, 5.4, 4000), (0.2, 5.4, 4000), (0, 5.6, 4000), (0.2, 5.6, 4000), (0, 5.8, 4000), (0.2, 5.8, 

In [67]:
kk_correct = [(0, 0, 4000), (0.2, 0, 4000), (0, 0.2, 4000), (0.2, 0.2, 4000), (0, 0.4, 4000), (0.2, 0.4, 4000), 
           (0, 0.6, 4000), (0.2, 0.6, 4000), (0, 0.8, 4000), (0.2, 0.8, 4000), (0, 1, 4000), (0.2, 1, 4000), 
           (0, 1.2, 4000), (0.2, 1.2, 4000), (0, 1.4, 4000), (0.2, 1.4, 4000), (0, 1.6, 4000), (0.2, 1.6, 4000), 
           (0, 1.8, 4000), (0.2, 1.8, 4000), (0, 2, 4000), (0.2, 2, 4000), (0, 2.2, 4000), (0.2, 2.2, 4000), 
           (0, 2.4, 4000), (0.2, 2.4, 4000), (0, 2.6, 4000), (0.2, 2.6, 4000), (0, 2.8, 4000), (0.2, 2.8, 4000), 
           (0, 3, 4000), (0.2, 3, 4000), (0, 3.2, 4000), (0.2, 3.2, 4000), (0, 3.4, 4000), (0.2, 3.4, 4000), 
           (0, 3.6, 4000), (0.2, 3.6, 4000), (0, 3.8, 4000), (0.2, 3.8, 4000), (0, 4, 4000), (0.2, 4, 4000), 
           (0, 4.2, 4000), (0.2, 4.2, 4000), (0, 4.4, 4000), (0.2, 4.4, 4000), (0, 4.6, 4000), (0.2, 4.6, 4000), 
           (0, 4.8, 4000), (0.2, 4.8, 4000), (0, 5, 4000), (0.2, 5, 4000), (0, 5.2, 4000), (0.2, 5.2, 4000), 
           (0, 5.4, 4000), (0.2, 5.4, 4000), (0, 5.6, 4000), (0.2, 5.6, 4000), (0, 5.8, 4000), (0.2, 5.8, 4000),
           (0, 6, 4000), (0.2, 6, 4000), (0, 6.2, 4000), (0.2, 6.2, 4000), (0, 6.4, 4000), (0.2, 6.4, 4000), 
           (0, 6.6, 4000), (0.2, 6.6, 4000), (0, 6.8, 4000), (0.2, 6.8, 4000), (0, 7, 4000), (0.2, 7, 4000), 
           (0, 7.2, 4000), (0.2, 7.2, 4000), (0, 7.4, 4000), (0.2, 7.4, 4000), (0, 7.6, 4000), (0.2, 7.6, 4000), 
           (0, 7.8, 4000), (0.2, 7.8, 4000), (0, 8, 4000), (0.2, 8, 4000)]

In [68]:
ArandANew = {}
for it in ArandA.keys():
    ArandANew[it] = {}
    for ind,k in enumerate(list(ArandA[it].keys())):
        if (np.abs(k[1] - kk_correct[ind][1])<0.01)&(np.abs(k[0] - kk_correct[ind][0])<0.0001): 
            ArandANew[it][kk_correct[ind]] = ArandA[it][k]
        else:
            print(k)

In [69]:
ArandANew[1].keys()

dict_keys([(0, 0, 4000), (0.2, 0, 4000), (0, 0.2, 4000), (0.2, 0.2, 4000), (0, 0.4, 4000), (0.2, 0.4, 4000), (0, 0.6, 4000), (0.2, 0.6, 4000), (0, 0.8, 4000), (0.2, 0.8, 4000), (0, 1, 4000), (0.2, 1, 4000), (0, 1.2, 4000), (0.2, 1.2, 4000), (0, 1.4, 4000), (0.2, 1.4, 4000), (0, 1.6, 4000), (0.2, 1.6, 4000), (0, 1.8, 4000), (0.2, 1.8, 4000), (0, 2, 4000), (0.2, 2, 4000), (0, 2.2, 4000), (0.2, 2.2, 4000), (0, 2.4, 4000), (0.2, 2.4, 4000), (0, 2.6, 4000), (0.2, 2.6, 4000), (0, 2.8, 4000), (0.2, 2.8, 4000), (0, 3, 4000), (0.2, 3, 4000), (0, 3.2, 4000), (0.2, 3.2, 4000), (0, 3.4, 4000), (0.2, 3.4, 4000), (0, 3.6, 4000), (0.2, 3.6, 4000), (0, 3.8, 4000), (0.2, 3.8, 4000), (0, 4, 4000), (0.2, 4, 4000), (0, 4.2, 4000), (0.2, 4.2, 4000), (0, 4.4, 4000), (0.2, 4.4, 4000), (0, 4.6, 4000), (0.2, 4.6, 4000), (0, 4.8, 4000), (0.2, 4.8, 4000), (0, 5, 4000), (0.2, 5, 4000), (0, 5.2, 4000), (0.2, 5.2, 4000), (0, 5.4, 4000), (0.2, 5.4, 4000), (0, 5.6, 4000), (0.2, 5.6, 4000), (0, 5.8, 4000), (0.2, 5.8, 

In [70]:
hf.saveVarSimple(ArandANew, filePathLoad)